In [1]:
using CSV
using DataFrames
using GLM
using Statistics
using Random
using Distributions
using Plots,StatsPlots
using DelimitedFiles
using SciPy
using PyPlot
using PyCall
using Normalization
using LinearAlgebra
@pyimport seaborn as sns
using Turing
using MCMCChains

# We need a logistic function, which is provided by StatsFuns.
using StatsFuns: logistic


In [2]:
np = pyimport("numpy")

PyObject <module 'numpy' from 'C:\\Users\\w4103\\.julia\\conda\\3\\x86_64\\Lib\\site-packages\\numpy\\__init__.py'>

In [3]:
# Load the data
x_data = CSV.read("x.csv", DataFrame)  # Load predictors
y_data = CSV.read("y.csv", DataFrame)  # Load target labels (assumes a single column)
X = Matrix(x_data)
y = y_data.y
X = hcat(ones(size(X,1)),X)

1000×21 Matrix{Float64}:
 1.0  -1.344    -1.23648  -1.03158  -1.18748  …  -0.42829  -1.2146   -0.19601
 1.0  -0.26535   2.24819  -0.02221  -1.18748     -0.42829   0.82332  -0.19601
 1.0   0.8133   -0.73867  -1.03158  -0.69556      2.33487   0.82332  -0.19601
 1.0  -1.344     1.75038  -0.02221  -0.20365      2.33487   0.82332  -0.19601
 1.0  -1.344     0.25695   0.98717   0.28826      2.33487   0.82332  -0.19601
 1.0   0.8133    1.25257  -0.02221  -0.69556  …   2.33487  -1.2146   -0.19601
 1.0   0.8133    0.25695  -0.02221  -0.20365     -0.42829   0.82332  -0.19601
 1.0  -0.26535   1.25257  -0.02221   0.78017     -0.42829  -1.2146   -0.19601
 1.0   0.8133   -0.73867  -0.02221  -1.18748     -0.42829   0.82332  -0.19601
 1.0  -0.26535   0.75476  -1.03158   0.28826     -0.42829   0.82332  -0.19601
 1.0  -0.26535  -0.73867  -0.02221   0.28826  …  -0.42829   0.82332  -0.19601
 1.0  -1.344     2.24819  -0.02221   1.27209     -0.42829   0.82332  -0.19601
 1.0  -0.26535  -0.73867  -0.02221  -1.

In [4]:
function train_test_split(X, y, test_size, shuffle,seed)
    # Set seed for reproducibility
    if seed !== nothing
        Random.seed!(seed)
    end

    n = length(y)  # Number of samples
    indices = collect(1:n)  # Create an index array

    # Shuffle indices if needed
    if shuffle
        shuffle!(indices)
    end

    # Compute train-test split index
    split_idx = Int(round(n * (1 - test_size)))

    # Split indices into training and testing sets
    train_indices = indices[1:split_idx]
    test_indices = indices[split_idx+1:end]

    # Extract train and test sets
    X_train, X_test = X[train_indices, :], X[test_indices, :]
    y_train, y_test = y[train_indices], y[test_indices]

    return X_train, X_test, y_train, y_test
end

train_test_split (generic function with 1 method)

In [5]:
rseed = 42

42

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.1, true, rseed)

println("Train size: ", size(X_train))
println("Test size: ", size(X_test))

Train size: (900, 21)
Test size: (100, 21)


In [7]:
cols_to_remove = [5, 7, 8, 11, 12, 13, 14, 15, 17, 18, 19]

11-element Vector{Int64}:
  5
  7
  8
 11
 12
 13
 14
 15
 17
 18
 19

In [8]:
X_train = X_train[:, setdiff(1:size(X_train, 2), cols_to_remove)]
X_test = X_test[:, setdiff(1:size(X_test, 2), cols_to_remove)]

100×10 Matrix{Float64}:
 1.0   0.8133   -1.23648  -1.03158  -1.07086  …  -0.5982    0.82332  -0.19601
 1.0   0.8133   -0.48976  -0.02221  -0.19964     -0.5982    0.82332  -0.19601
 1.0   0.8133   -0.73867  -0.02221  -0.66574     -0.5982   -1.2146   -0.19601
 1.0   0.8133    1.25257  -1.03158   1.07494     -0.5982    0.82332  -0.19601
 1.0   0.8133    1.00367   0.98717  -0.17979     -0.5982   -1.2146   -0.19601
 1.0   0.8133    2.746     1.99654   2.18505  …  -0.5982    0.82332  -0.19601
 1.0   0.8133   -0.73867  -1.03158  -0.71997     -0.5982   -1.2146   -0.19601
 1.0  -0.26535   0.75476  -0.02221  -0.09934     -0.5982    0.82332  -0.19601
 1.0  -0.26535  -0.24086  -1.03158  -0.84615     -0.5982   -1.2146   -0.19601
 1.0   0.8133   -0.73867  -1.03158  -0.44634     -0.5982    0.82332  -0.19601
 1.0  -0.26535   0.25695  -0.02221  -0.18121  …   0.68549  -1.2146   -0.19601
 1.0  -1.344    -0.24086  -0.02221  -0.31271     -0.5982    0.82332  -0.19601
 1.0  -0.26535  -1.23648   0.98717  -0.7

In [9]:
@model function logistic_regression(x, y, n, σ)
    intercept ~ Normal(0, σ)

    one ~ Normal(0, σ)
    two ~ Normal(0, σ)
    three ~ Normal(0, σ)
    four ~ Normal(0, σ)
    five ~ Normal(0, σ)
    six ~ Normal(0, σ)
    seven ~ Normal(0, σ)
    eight ~ Normal(0, σ)
    nine ~ Normal(0, σ)

    for i in 1:n
        v = logistic(intercept + one * x[i, 1] + two * x[i, 2] + three * x[i, 3]
            + four * x[i, 4] + five * x[i, 5] + six * x[i, 6]
            + seven * x[i, 7] + eight * x[i, 8] + nine * x[i, 9])
        y[i] ~ Bernoulli(v)
    end
end;

In [10]:
setprogress!(false)

[ Info: [Turing]: progress logging is disabled globally
[ Info: [AdvancedVI]: global PROGRESS is set as false


false

In [12]:
n, _ = size(X_train)

# Sample using NUTS.
m = logistic_regression(X_train, y_train, n, 1)
@time chain = sample(m, NUTS(), MCMCThreads(), 1_000, 1)

┌ Warning: Only a single thread available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC C:\Users\w4103\.julia\packages\AbstractMCMC\FSyVk\src\sample.jl:382
┌ Info: Found initial step size
└   ϵ = 0.2


 58.232444 seconds (45.52 M allocations: 3.789 GiB, 3.45% gc time, 74.83% compilation time)


Chains MCMC chain (1000×22×1 Array{Float64, 3}):

Iterations        = 501:1:1500
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 37.39 seconds
Compute duration  = 37.39 seconds
parameters        = intercept, one, two, three, four, five, six, seven, eight, nine
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std      mcse    ess_bulk   ess_tail      rhat    ⋯
      Symbol   Float64   Float64   Float64     Float64    Float64   Float64    ⋯

   intercept   -0.5551    0.7127    0.0436    266.0581   360.5326    1.0183    ⋯
         one   -0.4683    0.7103    0.0425    275.4556   395.9872    1.0181    ⋯
         two   -0.6406    0.0828    0.0028    834.4564   709.7250    0.9995    ⋯
       three    0.2774    0.0976    0.0037    698.1568   594.2166    0.9990    ⋯
        f